OpenTire Moment Method Example
=================
Draws a constant velocity force-moment diagram of a bicycle using the OpenTire library

Import OpenTire and other libraries used in this demonstration

In [ ]:
from opentire import OpenTire
from opentire.Core import TireState

import numpy as np
import matplotlib.pyplot as plt

Define the vehicle class

In [ ]:
class Vehicle():
    def __init__(self):
        self._mass = 1000
        self._wb = 1
        self._wd = 0.5
        self._ft = None
        self._rt = None
    
    @property
    def mass(self):
        return self._mass
    
    @mass.setter
    def mass(self, value):
        if value <= 0:
            raise ValueError('Mass must be greater than zero')
        self._mass = value
    
    @property
    def wheelbase(self):
        return self._wb
    
    @wheelbase.setter
    def wheelbase(self, value):
        if value <= 0:
            raise ValueError('Wheelbase must be greater than zero')
        self._wb = value
        
    @property
    def weight_dist(self):
        return self._wd
    
    @weight_dist.setter
    def weight_dist(self, value):
        if value >= 1 or value <= 0:
            raise ValueError('Weight distribution must be a ratio between 0 and 1')
        self._wd = value
        
    @property
    def front_tire(self):
        return self._ft
    
    @front_tire.setter
    def front_tire(self, value):
        if not isinstance(value, TireModelBase) and value is not None:
            raise TypeError('Front tire must be a OpenTire model')
        self._ft = value
        
    @property
    def rear_tire(self):
        return self._rt
    
    @front_tire.setter
    def rear_tire(self, value):
        if not isinstance(value, TireModelBase) and values is not None:
            raise TypeError('Rear tire must be a OpenTire model')
        self._rt = value
        
    @property
    def length_a(self):
        return self.wheelbase * (1 - self.weight_dist)
    
    @property
    def length_b(self):
        return self.wheelbase * self.weight_dist

Define the Moment Method class

In [ ]:
class MomentMethodSolver():
    def __init__(self):
        self._vehicle = None
        self._beta = np.linspace(-12, 12, 25) * 3.14 / 180
        self._delta = np.linspace(-12, 12, 25) * 3.14 / 180
        self._velocity = 36
    
    @property
    def vehicle(self):
        return self._vehicle
    
    @vehicle.setter
    def vehicle(self, value):
        if not isinstance(value, Vehicle) and value is not None:
            raise TypeError('Solver vehicle must be of Vehicle type')
        self._vehicle = value
        
    @property
    def beta_range(self):
        return self._beta
    
    @beta_range.setter
    def beta_range(self, value):
        self._beta = value
        
    @property
    def delta_range(self):
        return self._delta
    
    @delta_range.setter
    def delta_range(self, value):
        self._delta = value   
    
    @property
    def velocity(self):
        return self._velocity
    
    @velocity.setter
    def velocity(self, value):
        self._velocity = value
    
    def solve(self):
        fy = np.empty([len(self.beta_range), len(self.delta_range)])
        mz = np.empty([len(self.beta_range), len(self.delta_range)])
        initial_guess = (0, 0)
        
        for i, beta in enumerate(self.beta_range):
            for j, delta in enumerate(self.delta_range):
                # Use previous solution as a guess
                if j > 0: initial_guess = self._invertSolution(fy[i][j-1], mz[i][j-1])
                elif i > 0: initial_guess = self._invertSolution(fy[i-1][j], mz[i-1][j])
                else: initial_guess = (0, 0)
                
                result = self._solve(beta, delta, initial_guess)
                fy[i][j] = result[0]
                mz[i][j] = result[1]
                
        return (fy, mz)
    
    def _solve(self, beta, delta, initial_guess = (0, 0)):
        state = TireState()
        state['FZ'] = 1500
        state['IA'] = 0.0
        state['SR'] = 0.0
        state['SA'] = 0.0
        state['FY'] = 0.0
        state['V'] = 10.0
        state['P'] = 260000
        
        MAX_ITER = 100
        n = 0
        error = 9999
        tolerance = 0.1
        yaw_velocity = 0
        
        front_force = initial_guess[0]
        rear_force = initial_guess[1]
        
        while (n < MAX_ITER and abs(error) > tolerance):            
            # Yaw rate
            yaw_velocity = (front_force + rear_force) / (self.vehicle.mass * self.velocity)
            error = front_force + rear_force
            
            # Slip Angles
            sa_front = beta - delta + yaw_velocity * self.vehicle.length_a / self.velocity
            sa_rear = beta - yaw_velocity * self.vehicle.length_b / self.velocity

            # Front Tire
            state['SA'] = sa_front
            state['FZ'] = 0.5 * 9.81 * self.vehicle.mass * self.vehicle.weight_dist
            self.vehicle.front_tire.solve(state)

            front_force = state['FY']
            state['SA'] = -sa_front
            self.vehicle.front_tire.solve(state)
            front_force -= state['FY']

            # Rear Tire
            state['SA'] = sa_rear
            state['FZ'] = 0.5 * 9.81 * self.vehicle.mass * (1 - self.vehicle.weight_dist)
            self.vehicle.rear_tire.solve(state)

            rear_force = state['FY']
            state['SA'] = -sa_rear
            self.vehicle.rear_tire.solve(state)
            rear_force -= state['FY']
            
            error -= front_force + rear_force
            n += 1
            
        return (front_force + rear_force,
                front_force * self.vehicle.length_a - rear_force * self.vehicle.length_b)
    
    def _invertSolution(self, lateral_force, yaw_moment):
        front_force = (1 / (self.vehicle.length_a + self.vehicle.length_b)) * (self.vehicle.length_b * lateral_force
                                                                               + yaw_moment)
        rear_force = (1 / (self.vehicle.length_a + self.vehicle.length_b)) * (self.vehicle.length_a * lateral_force 
                                                                              - yaw_moment)
            
        return (front_force, rear_force)

Run simulation

In [ ]:
openTire = OpenTire()

myVehicle = Vehicle()
myVehicle.mass = 1250  # kg
myVehicle.wheelbase = 2.4  # m
myVehicle.weight_dist = 0.47  # ratio
myVehicle.front_tire = openTire.createmodel('PAC2002')
myVehicle.rear_tire = openTire.createmodel('PAC2002')

solver = MomentMethodSolver()
solver.vehicle = myVehicle
solver.beta = np.linspace(-15, 16, 31) * 3.14 / 180
solver.delta = np.linspace(-15, 16, 31) * 3.14 / 180
solver.velocity = 70 / 3.6

force_moments = solver.solve()
lateral_accel = force_moments[0] / myVehicle.mass / 9.81
yaw_moment = force_moments[1]

plt.plot(lateral_accel[:][:], yaw_moment[:][:], color='black')
plt.plot(np.transpose(lateral_accel[:][:]), np.transpose(yaw_moment[:][:]), color='red')

plt.grid()
plt.xlabel("Lateral Acceleration [g]")
plt.ylabel("Yaw Moment [Nm]")

plt.show()